In [1]:
import os
import openai
import dotenv

dotenv.load_dotenv()

True

In [2]:
# ! pip install llama-index
# ! pip install pypdf

In [3]:
from llama_index import SimpleDirectoryReader
from llama_index import Document

documents = SimpleDirectoryReader(
    input_files=["./data/dynalist-2023-12-8.txt"]
).load_data()

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [4]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

1 

<class 'llama_index.schema.Document'>
Doc ID: 1fda7786-d1f2-4955-8d0f-aea6fcf7b7cf
Text: ﻿A Comprehensive Mechanistic Interpretability Explainer &
Glossary     __Written by [Neel Nanda](https://www.neelnanda.io/) to
accompany the [TransformerLens library](https://github.com/neelnanda-
io/TransformerLens) for mechanistic interpretability of language
models. I love hearing if people find my work useful, please [reach
out](mailto:neelna...


In [6]:
len(document.text)

255302

In [7]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

In [8]:
query_engine = index.as_query_engine()

In [9]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
print(str(response))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


When finding projects to build your experience, there are several steps you can take. First, you can start by identifying your areas of interest and expertise. This will help you narrow down the types of projects you want to work on. Next, you can research and explore different project ideas within your chosen field. Look for projects that align with your goals and offer opportunities for growth and learning. Once you have a list of potential projects, evaluate them based on feasibility, resources required, and potential impact. Choose projects that are challenging but achievable within your current skill set. Finally, prioritize and plan your projects, setting clear goals and timelines for each one. This will help you stay organized and focused as you work on building your experience.


In [10]:
eval_questions = [
    'What are the keys to building a career in AI?', 
    'How can teamwork contribute to success in AI?', 
    'What is the importance of networking in AI?', 
    'What are some good habits to develop for a successful career?', 
    'How can altruism be beneficial in building a career?',
    'What is imposter syndrome and how does it relate to AI?', 
    'Who are some accomplished individuals who have experienced imposter syndrome?', 
    'What is the first step to becoming good at AI?', 
    'What are some common challenges in AI?', 
    'Is it normal to find parts of AI challenging?', 
    'What is the right AI job for me?']

In [19]:
eval_questions = [
    "What is mechanistic interpretability?",
    "Is mechanistic interpretability appliable to real world?",
    "What is superposition and how can I understand it?",
    "How activation functions relate to interpretability of a model?",
    "What is feature?",
    "What is the difference between activation and a feature?",
    "What is a circuit?",
    "What is an Induction Head?",
    "How Induction Head was found?",
    "Why induction head is so important?",
    "What are other examples of found circuits except for Induction Head?"
    "What is sparsity?",
    "What is the phase transition?"
]

In [20]:
# ! pip install trulens-eval

In [21]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

In [22]:
import numpy as np
from trulens_eval import (
    Feedback,
    TruLlama,
    OpenAI
)

openai = OpenAI()

from trulens_eval.feedback import Groundedness

qa_relevance = (
    Feedback(openai.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input_output()
)

qs_relevance = (
    Feedback(openai.relevance_with_cot_reasons, name = "Context Relevance")
    .on_input()
    .on(TruLlama.select_source_nodes().node.text)
    .aggregate(np.mean)
)

#grounded = Groundedness(groundedness_provider=openai, summarize_provider=openai)
grounded = Groundedness(groundedness_provider=openai)

groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name="Groundedness")
        .on(TruLlama.select_source_nodes().node.text)
        .on_output()
        .aggregate(grounded.grounded_statements_aggregator)
)

feedbacks = [qa_relevance, qs_relevance, groundedness]

def get_prebuilt_trulens_recorder(query_engine, app_id):
    tru_recorder = TruLlama(
        query_engine,
        app_id=app_id,
        feedbacks=feedbacks
        )
    return tru_recorder

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [23]:
tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

In [24]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [25]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [26]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_1fe1519f0317d831b8ebb8e48a897eac,"""What is mechanistic interpretability?""","""Mechanistic interpretability refers to the fi...",-,"{""record_id"": ""record_hash_1fe1519f0317d831b8e...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-08T08:30:38.178408"", ""...",2023-12-08T08:30:41.300499,1.0,0.45,1.0,[{'args': {'prompt': 'What is mechanistic inte...,[{'args': {'prompt': 'What is mechanistic inte...,[{'args': {'source': '﻿A Comprehensive Mechani...,3,2165,0.003280
1,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_eb35b35db9c92f9f946ac43e68375643,"""Is mechanistic interpretability appliable to ...","""Mechanistic interpretability is applicable to...",-,"{""record_id"": ""record_hash_eb35b35db9c92f9f946...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-08T08:30:41.457956"", ""...",2023-12-08T08:30:44.644291,1.0,0.40,NaN,[{'args': {'prompt': 'Is mechanistic interpret...,[{'args': {'prompt': 'Is mechanistic interpret...,NaN,3,2180,0.003308
2,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_73f41e813e7103e6a2f80bc98ab6818f,"""What is superposition and how can I understan...","""Superposition refers to the phenomenon where ...",-,"{""record_id"": ""record_hash_73f41e813e7103e6a2f...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-08T08:30:44.795600"", ""...",2023-12-08T08:30:51.659492,0.9,NaN,NaN,[{'args': {'prompt': 'What is superposition an...,NaN,NaN,6,2315,0.003585
3,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_6576f0e9e9542cded3f597fafde7d24d,"""How activation functions relate to interpreta...","""Activation functions play a role in the inter...",-,"{""record_id"": ""record_hash_6576f0e9e9542cded3f...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-08T08:30:51.809913"", ""...",2023-12-08T08:30:55.995250,NaN,NaN,NaN,NaN,NaN,NaN,4,2166,0.003297


In [27]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.68.103:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>